## PoseVis Data Quality

Runs a series of benchmarks and reports data quality metrics such as dropped frame percentage, latency, and jitter.

### Install py-cpuinfo for System Info

In [19]:
!pip install py-cpuinfo


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import cpuinfo

info = cpuinfo.get_cpu_info()
print(f"Python version: {info['python_version']}")
print(f"CPU: {info['brand_raw']}")

Python version: 3.10.8.final.0 (64 bit)
CPU: Intel(R) Core(TM) i7-6700K CPU @ 4.00GHz


### Run Benchmarks

Devices tested are: (0) Logitech C270 Webcam, and (1) VUPUMER Webcam

In [12]:
import os
import pose_vis.pose_vis

from dataclasses import dataclass
from typing import List, Tuple
from pose_vis.extension import PoseVisExtension
from pose_vis.runner import PoseVisConfig
from pose_vis.runners.benchmark_runner import BenchmarkRunner, BenchmarkRunnerConfig
from pose_vis.extensions.hands import HandsExtension

@dataclass
class BenchmarkConfig():
    extensions: List[PoseVisExtension]
    logging: bool
    sources: List[int]
    resolution: Tuple[int, int, int]
    runtime: int

benchmarks = [
    # Single camera, 1280x720x30
    BenchmarkConfig([], False, [0], (1280, 720, 30), 60),
    # Single camera, 1280x720x30, hand tracking
    BenchmarkConfig([HandsExtension()], False, [0], (1280, 720, 30), 60),
    # Two cameras, 1280x720x30
    BenchmarkConfig([], False, [0, 1], (1280, 720, 30), 60),
    # Two cameras, 1280x720x30, hand tracking
    BenchmarkConfig([HandsExtension()], False, [0, 1], (1280, 720, 30), 60),
]

for benchmark in benchmarks:
    config = PoseVisConfig(
        extensions = benchmark.extensions,
        log_directory = f"webcam{os.sep}logs",
        log_name = "benchmark",
        enable_logging = benchmark.logging,
        display_framerate = 0,
        stats_history_size = 0)

    resolutions = [benchmark.resolution for _ in range(len(benchmark.sources))]
    output_name = f"benchmark_{len(benchmark.sources)}_sources_{resolutions[0][0]}x{resolutions[0][1]}x{resolutions[0][2]}"

    if len(benchmark.extensions) > 0:
        ext_names = ""
        for i in range(len(benchmark.extensions)):
            sep = "_" if i > 0 else ""
            ext_names += f"{sep}{benchmark.extensions[i].__class__.__name__}"
        output_name = f"{output_name}_{ext_names}"
    if benchmark.logging:
        output_name = f"{output_name}_logging"
    
    runner_config = BenchmarkRunnerConfig(
        sources = benchmark.sources,
        resolutions = resolutions,
        output_path = f"webcam{os.sep}logs",
        output_name = output_name,
        run_time = 60)
    runner = BenchmarkRunner(config, runner_config)

    print(f"### Running benchmark: {output_name}")
    runner.build()
    runner.run()
    print(f"### Benchmark complete: {output_name}")

INFO:pose_vis.runner: building graph
INFO:pose_vis.runner: logging directory is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runners.benchmark_runner: benchmark output path is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runner: running graph


### Running benchmark: benchmark_1_sources_1280x720x30


INFO:pose_vis.runner: building graph
INFO:pose_vis.runner: enabling extension: HandsExtension
INFO:pose_vis.runner: logging directory is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runners.benchmark_runner: benchmark output path is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runner: running graph


### Benchmark complete: benchmark_1_sources_1280x720x30
### Running benchmark: benchmark_1_sources_1280x720x30_HandsExtension


INFO:pose_vis.runner: building graph
INFO:pose_vis.runner: logging directory is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runners.benchmark_runner: benchmark output path is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runner: running graph


### Benchmark complete: benchmark_1_sources_1280x720x30_HandsExtension
### Running benchmark: benchmark_2_sources_1280x720x30


INFO:pose_vis.runner: building graph
INFO:pose_vis.runner: enabling extension: HandsExtension
INFO:pose_vis.runner: logging directory is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runners.benchmark_runner: benchmark output path is c:\Users\das\Desktop\labgraph\devices\webcam\logs
INFO:pose_vis.runner: running graph


### Benchmark complete: benchmark_2_sources_1280x720x30
### Running benchmark: benchmark_2_sources_1280x720x30_HandsExtension
### Benchmark complete: benchmark_2_sources_1280x720x30_HandsExtension


### Display Results

In [13]:
import json
import statistics
from pose_vis.utils import absolute_path
from typing import Dict, Union, List, Tuple

json_files = [_file for _file in os.listdir(absolute_path(f"webcam{os.sep}logs")) if _file.endswith(".json")]

for filename in json_files:
    timings: Dict[str, Union[float, int, List[Tuple[float, float, float]]]] = {}

    with open(absolute_path(f"webcam{os.sep}logs{os.sep}{filename}")) as _file:
        timings = json.loads(_file.read())

    print(f"## Benchmark: {filename.removesuffix('.json')}")
    print(f"# runtime: {timings['runtime']:.2f}s")

    expected_frames = timings["runtime"] * timings["target_fps"]
    # Estimate of how many frames were dropped while capturing the source
    source_received_frames_pct = timings["frame_index"] / expected_frames
    print(f"# dropped: {(100 - (source_received_frames_pct * 100)):.2f}%")

    latency: List[float] = []
    desync: List[List[float]] = []
    # device time = system time at frame capture from device
    # All times are captured with time.perf_counter()
    # times: List[(device time, receive time)]
    for i in range(1, len(timings["times"])):
        rel_device = timings["times"][i][0] - timings["times"][0][0]
        rel_receive = timings["times"][i][1] - timings["times"][0][1]
        # TODO: this may not be correct
        # `rel_device` can be greater than `rel_receive`, for now we just take the absolute value
        latency.append(abs(rel_receive - rel_device))

        if len(timings["sync"]) > 0:
            if i == 1:
                desync = [[] for j in range(len(timings["sync"][i]))]

            for j in range(len(timings["sync"][i])):
                desync[j].append(abs(timings["times"][i][0] - timings["sync"][i][j]))
    
    print(f"# latency: {statistics.median(latency) * 1000:.2f}ms")
    print(f"# jitter: {statistics.stdev(latency) * 1000:.2f}ms")
    print("# [desync]")
    for i, li in enumerate(desync):
        print(f"# from source 0 to source {i + 1}: {statistics.median(desync[i]) * 1000:.2f}ms, jitter: {statistics.stdev(desync[i]) * 1000:.2f}ms")
    print(" ")

## Benchmark: benchmark_1_sources_1280x720x30
# runtime: 60.55s
# dropped: 0.75%
# latency: 16.69ms
# jitter: 5.04ms
# [desync]
 
## Benchmark: benchmark_1_sources_1280x720x30_HandsExtension
# runtime: 60.45s
# dropped: 0.64%
# latency: 5.17ms
# jitter: 4.14ms
# [desync]
 
## Benchmark: benchmark_2_sources_1280x720x30
# runtime: 60.42s
# dropped: 0.64%
# latency: 13.33ms
# jitter: 11.09ms
# [desync]
# from source 0 to source 1: 2.43ms, jitter: 11.02ms
 
## Benchmark: benchmark_2_sources_1280x720x30_HandsExtension
# runtime: 60.46s
# dropped: 28.10%
# latency: 4.94ms
# jitter: 5.80ms
# [desync]
# from source 0 to source 1: 0.11ms, jitter: 6.10ms
 
